In [2]:
from gpcam import GPOptimizer

import torch
from torch import nn

import numpy as np
from numpy.random import default_rng

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import os
import csv

import random

# Import Data

In [3]:
energy_data = np.load("/data/Synthetic Data Generation_1/my_synthetic_energy.npy")
cycle_number = np.load("/data/Synthetic Data Generation_1/my_synthetic_cycleNum.npy")


label_size = 30

num_of_datasets = 50

considered_batteries = ([5546, 9477, 2231, 4437, 7059, 5259, 8330, 1068, 8214, 5888, 3275, 6845, 7671, 
                         299, 5038, 3503, 8673, 2236, 3644, 4980, 993, 7545, 654, 1418, 6090, 7936, 8792, 
                         6910, 2933, 2382, 9730, 8476, 1882, 7986, 7091, 4813, 3086, 3908, 1539, 8567, 2152, 
                         5738, 8646, 9692, 2661, 6766, 7230, 512, 758, 2881])


print(considered_batteries)


plt.figure(figsize = (20,10))
for i in considered_batteries: plt.scatter(cycle_number,energy_data[int(i)])

plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("Quantity of Interest",fontsize=label_size)
plt.show()

print("max y: ", np.max(energy_data))

# Initializing the data to fit the GP model
data_size = num_of_datasets

# All Data
x_data = np.tile(cycle_number, data_size).reshape(-1, 1) # repeat cycle 20 times to create x_data
y_data = np.vstack(energy_data[considered_batteries, :].T).reshape(-1, 1)
x_pred = np.linspace(0,1000,1001).reshape(-1,1)


print("x data: ", x_data.shape)
print("y data: ", y_data.shape)
        
    
plt.scatter(x_data,y_data)


[5546, 9477, 2231, 4437, 7059, 5259, 8330, 1068, 8214, 5888, 3275, 6845, 7671, 299, 5038, 3503, 8673, 2236, 3644, 4980, 993, 7545, 654, 1418, 6090, 7936, 8792, 6910, 2933, 2382, 9730, 8476, 1882, 7986, 7091, 4813, 3086, 3908, 1539, 8567, 2152, 5738, 8646, 9692, 2661, 6766, 7230, 512, 758, 2881]
max y:  529.5870098158235
x data:  (2500, 1)
y data:  (2500, 1)


# Creating the Subfolder in Results

In [1]:
# Specify the path for the new folder
new_folder_path = f"/results/Methods Figures"

# Create the folder
os.makedirs(new_folder_path, exist_ok=True)

NameError: name 'os' is not defined

# Deep Kernel Learning

In [4]:
import torch
from torch import nn

# All remaining code asssumes that the NN architecture is made up of two hidden layers and same number of nodes
# If other architectures are used, the indices of the hyperparameters and their boudsn need to be changed accordingly
# Number of nodes can be varied as the user prefer

class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.nodes_num = 5

        # Inputs to hidden layer linear transformation
        self.layer1 = nn.Linear(1, self.nodes_num)
        self.layer2 = nn.Linear(self.nodes_num, self.nodes_num)
        self.layer3 = nn.Linear(self.nodes_num, 1)

    def forward(self, x):
        # Pass the input tensor through each of our operations
        #print(x)
        x = torch.Tensor(x)
        x = torch.nn.functional.relu(self.layer1(x))
        x = torch.nn.functional.relu(self.layer2(x))
        x = torch.nn.functional.relu(self.layer3(x))
        return x.detach().numpy()

    def set_weights(self,w1,w2,w3):
      with torch.no_grad():
        self.layer1.weight = nn.Parameter(torch.from_numpy(w1).float())
        self.layer2.weight = nn.Parameter(torch.from_numpy(w2).float())
        self.layer3.weight = nn.Parameter(torch.from_numpy(w3).float())

    def set_biases(self,b1,b2,b3):
      with torch.no_grad():
        self.layer1.bias = nn.Parameter(torch.from_numpy(b1).float())
        self.layer2.bias = nn.Parameter(torch.from_numpy(b2).float())
        self.layer3.bias = nn.Parameter(torch.from_numpy(b3).float())

    def get_weights(self):
        return self.layer1.weight, self.layer2.weight, self.layer3.weight
    def get_biases(self):
        return self.layer1.bias, self.layer2.bias, self.layer3.bias

n = Network()


# Defining GP Components

In [5]:
# For the squared Exponential Function
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d)

# For the Noise
def s(x, my_slope, my_pow, my_intercept):
    o = my_slope * x**my_pow + my_intercept
    return o

'''

def my_noise(x,hps,obj):

    total_num_of_NN_hps = obj.args[1]

    my_slope     = hps[total_num_of_NN_hps+1]
    my_pow       = hps[total_num_of_NN_hps+2]
    my_intercept = hps[total_num_of_NN_hps+3]

    noise = np.identity(len(x)) * np.outer(s(x,my_slope,my_pow,my_intercept),s(x,my_slope,my_pow,my_intercept))

    return noise


# Kernel Function
def kernel_nn(x1,x2,hps,obj):

    nodes_num           = obj.args[0]
    total_num_of_NN_hps = obj.args[1]

    
   
    # NN
    n.set_weights(hps[0:nodes_num].reshape(nodes_num,1),
                  hps[nodes_num:nodes_num**2+nodes_num].reshape(nodes_num,nodes_num),
                  hps[nodes_num**2+nodes_num:nodes_num**2+2*nodes_num].reshape(1,nodes_num))

    n.set_biases(hps[nodes_num**2+2*nodes_num:nodes_num**2+3*nodes_num].reshape(nodes_num),
                 hps[nodes_num**2+3*nodes_num:nodes_num**2+4*nodes_num].reshape(nodes_num),
                 np.array([hps[nodes_num**2+4*nodes_num]]))

    x1_nn = n.forward(x1).reshape(-1,1)
    x2_nn = n.forward(x2).reshape(-1,1)
    d = get_distance_matrix(x1_nn,x2_nn)

    # Main Function
    k = hps[total_num_of_NN_hps] * obj.squared_exponential_kernel(d,200)
    return k


################################################################################

# Mean function: Two-Element piecewise function
def mean2(x,hps,obj):

    total_num_of_NN_hps = obj.args[1]

    x0 = hps[total_num_of_NN_hps+4]
    
    m1 = hps[total_num_of_NN_hps+5]
    m2 = hps[total_num_of_NN_hps+6]

    b1  = 500
    b2 = (m1 - m2) * x0 + b1

    x = x[:,0]

    y = np.where(x <= x0, m1*x + b1, m2*x + b2)
                
    return y


def my_noise(x,hps,obj):

    total_num_of_NN_hps = obj.args[1]

    my_slope     = hps[total_num_of_NN_hps+1]
    my_pow       = hps[total_num_of_NN_hps+2]
    my_intercept = hps[total_num_of_NN_hps+3]

    noise = np.identity(len(x)) * np.outer(s(x,my_slope,my_pow,my_intercept),s(x,my_slope,my_pow,my_intercept))

    return noise
'''

def my_noise(x,hps,obj):

    my_slope     = hps[1]
    my_pow       = hps[2]
    my_intercept = hps[3]

    noise = np.identity(len(x)) * np.outer(s(x,my_slope,my_pow,my_intercept),s(x,my_slope,my_pow,my_intercept))

    return noise

# Kernel Function
def kernel_nn(x1,x2,hps,obj):

    nodes_num           = obj.args[0]
    total_num_of_NN_hps = obj.args[1]

    
    trained_NN_hps = np.array([-0.349,  0.461, -0.678,  0.604,  0.747,  0.827,  0.282,  0.126,  0.572, -0.313,
                   -0.463, -0.174, -0.619,  0.701, -0.109,  0.048, -0.318, -0.367, -0.499, -0.321,
                    0.083,  0.668, -0.209,  0.552, -0.355,  0.192,  0.772, -0.739,  0.959, -0.772,
                   -0.713, -0.817, -0.014,  0.077,  0.381, -196.313,  126.321,   45.903, -216.376,  
                    126.7, -86.184, -60.536, -108.548, 67.332,  162.204,  -36.093])
    
    # NN
    n.set_weights(trained_NN_hps[0:nodes_num].reshape(nodes_num,1),
                  trained_NN_hps[nodes_num:nodes_num**2+nodes_num].reshape(nodes_num,nodes_num),
                  trained_NN_hps[nodes_num**2+nodes_num:nodes_num**2+2*nodes_num].reshape(1,nodes_num))

    n.set_biases(trained_NN_hps[nodes_num**2+2*nodes_num:nodes_num**2+3*nodes_num].reshape(nodes_num),
                 trained_NN_hps[nodes_num**2+3*nodes_num:nodes_num**2+4*nodes_num].reshape(nodes_num),
                 np.array([trained_NN_hps[nodes_num**2+4*nodes_num]]))

    x1_nn = n.forward(x1).reshape(-1,1)
    x2_nn = n.forward(x2).reshape(-1,1)
    d = get_distance_matrix(x1_nn,x2_nn)

    # Main Function
    k = hps[0] * obj.squared_exponential_kernel(d,200)
    return k


################################################################################

# Mean function: Two-Element piecewise function
def mean2(x,hps,obj):

    total_num_of_NN_hps = obj.args[1]

    x0 = hps[4]
    
    m1 = hps[5]
    m2 = hps[6]

    b1  = 500
    b2 = (m1 - m2) * x0 + b1

    x = x[:,0]

    y = np.where(x <= x0, m1*x + b1, m2*x + b2)
                
    return y

# Training the Model

In [6]:
x_pred_original = np.linspace(0,1000,1001).reshape(-1,1)

nodes_num = n.nodes_num

total_num_of_NN_hps = nodes_num**2 + 4*nodes_num + 1  # Depends on the number of layers used
num_of_other_hps = 7                                  # Depends on Kernel, noise and mean functions


NN_weights_initial_hps = np.random.uniform(-1, 1,nodes_num**2+2*nodes_num)  
NN_biases_initial_hps = np.random.uniform(-100,100,total_num_of_NN_hps-(nodes_num**2+2*nodes_num))


other_init_hps = np.array( [ 1.04806e+02,  1.00000e-03,  1.59400e+00,  1.58100e+00,  5.08026e+02,
 -5.00000e-02, -2.53000e-01])
   
    
#init_hyperparameters = np.concatenate([NN_weights_initial_hps,NN_biases_initial_hps,other_init_hps])


init_hyperparameters = other_init_hps

print("x data: ", x_data.shape)
print("y data: ", y_data.shape)



my_gpNN = GPOptimizer(x_data,y_data,
            #init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
            init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
            #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
            compute_device='cpu', 
            gp_kernel_function=kernel_nn, 
            gp_kernel_function_grad=None, 
            gp_mean_function=mean2, 
            gp_mean_function_grad=None,
            gp_noise_function=my_noise,
            normalize_y=False,
            sparse_mode=False,
            gp2Scale = False,
            store_inv=False, 
            ram_economy=False, 
            args= np.array([nodes_num,total_num_of_NN_hps]))

# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((num_of_other_hps,2))

# Kernel Sq Exp 
bounds[0] = np.array([100.,10000.])                             # Kernel Variance
#bounds[total_num_of_NN_hps+1] = np.array([2.,300.])                           # Kernel Lengthscale

# Noise
bounds[1] = np.array([1e-5,1.])                            # Noise Slope
bounds[2] = np.array([1.,5.])                            # Noise Power
bounds[3] = np.array([0.,3.])                              # Noise Intercept
# Mean
bounds[4] = np.array([200.,700.])                          # Mean Piecewise Intersection point
bounds[5] = np.array([-1e-1,-1e-3])                        # Mean Slope 1
bounds[6] = np.array([-5e-1,-1e-3])                        # Mean Slope 2


'''
bounds = np.empty((total_num_of_NN_hps + num_of_other_hps,2))

# NN
bounds[0:nodes_num**2+2*nodes_num] = np.array([-1.,1.])                      # Weights NN: Define spread and shift in output
bounds[nodes_num**2+2*nodes_num:total_num_of_NN_hps] = np.array([-100.,100.])#Biases of NN: Define shift in output

# Kernel Sq Exp 
bounds[total_num_of_NN_hps] = np.array([1000.,10000.])                             # Kernel Variance
#bounds[total_num_of_NN_hps+1] = np.array([2.,300.])                           # Kernel Lengthscale

# Noise
bounds[total_num_of_NN_hps+1] = np.array([1e-5,1.])                            # Noise Slope
bounds[total_num_of_NN_hps+2] = np.array([1.,5.])                            # Noise Power
bounds[total_num_of_NN_hps+3] = np.array([0.,3.])                              # Noise Intercept
# Mean
bounds[total_num_of_NN_hps+4] = np.array([200.,400.])                          # Mean Piecewise Intersection point
bounds[total_num_of_NN_hps+5] = np.array([-2e-1,-1e-2])                        # Mean Slope 1
bounds[total_num_of_NN_hps+6] = np.array([-2e-1,-1e-2])                        # Mean Slope 2
'''


#my_gpNN.train(hyperparameter_bounds=bounds, method = "global")


print("Training is Done")


x data:  (2500, 1)
y data:  (2500, 1)
Training is Done


In [9]:
x_pred_nn = n.forward(x_pred)

label_size = 30
plt.figure(figsize = (10,10))
plt.plot(x_pred_original,x_pred_nn,linewidth = 7,color = 'blue')
plt.xlabel("Original Domain ($\mathcal{X}$)",fontsize=label_size)
plt.ylabel("Transformed Domain ($\mathcal{X}^*$)",fontsize=label_size)
#plt.title("Cycle Number",fontsize=label_size)
plt.ylim([0,100])
plt.yticks([0,25,50,75,100])
plt.xlim([0,1000])
plt.xticks([0,250,500,750,1000])
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.savefig('/results/Methods Figures/NN Transformation.pdf',  bbox_inches='tight') # saving plot with a unique name 
